# Hyperparameter Tuning (Optimization)

A **hyperparameter**, such as $alpha$ in Ridge/lasso regression, $k$ in 
k-Nearest Neighbors or $C$ in  logistic regression, is a parameter whose value is not learned but set before the model fitting. On the other hand, the values of other parameters are derived via machine learning. 

There are several approaches to choose the optimal hyperparameters. Some of the are listed below.
 - Grid search
 - Random search
 - Bayesian optimization
 - Gradient-based optimization
 - Evolutionary optimization


In [18]:
import pandas as pd
import numpy as np

In [7]:
df = pd.read_csv("datasets/diabetes.csv")

>Note: I'll keep the EDA short, since the focus of this analysis is not exploring the data.

In [9]:
df.head(3)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1


In [10]:
## Fitting a Model

In [15]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [14]:
X = df.drop('Outcome', axis = 1).values # features
y = df['Outcome'].values # target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.4, random_state=42)

In [22]:
# Create the classifier
logreg = LogisticRegression()

# Fit the classifier to the training data
logreg.fit(X_train, y_train)

# Predict the labels of the test set
y_pred = logreg.predict(X_test)

## Hyperparameter tuning with Grid Search (GridSearchCV)

"Grid Search is an exhaustive searching through a manually specified subset of the hyperparameter space."

In [20]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

In [21]:
# Setup the hyperparameter grid
c_space = np.logspace(-5, 8, 15)
param_grid = {'C': c_space}

# Instantiate a logistic regression classifier
logreg = LogisticRegression()

# Instantiate the GridSearchCV object
logreg_cv = GridSearchCV(logreg, param_grid, cv=5)

# Fit it to the data
logreg_cv.fit(X, y)

# Print the tuned parameters and score
print("Tuned Logistic Regression Parameters: {}".format(logreg_cv.best_params_)) 
print("Best score is {}".format(logreg_cv.best_score_))

Tuned Logistic Regression Parameters: {'C': 268.2695795279727}
Best score is 0.7708333333333334


In [ ]:
## Hyperparameter tuning with Random Search (RandomizedSearchCV)

When we are dealing with multiple hyperparameters, Grid Search method may be computationally expensive. Random Search algorithm can outperform Grid search and helps us mainly in cases when we have a large hyperparameter space. 

In [23]:
from scipy.stats import randint
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV

In [25]:
# Setup the parameters and distributions to sample from
param_dist = {"max_depth": [3, None],
              "max_features": randint(1, 9),
              "min_samples_leaf": randint(1, 9),
              "criterion": ["gini", "entropy"]}

# Instantiate a Decision Tree classifier
tree = DecisionTreeClassifier()

# Instantiate the RandomizedSearchCV object
tree_cv = RandomizedSearchCV(tree, param_dist, cv=5)

# Fit it to the data
tree_cv.fit(X, y)

# Print the tuned parameters and score
print("Tuned Decision Tree Parameters: {}".format(tree_cv.best_params_))
print("Best score is {}".format(tree_cv.best_score_))

Tuned Decision Tree Parameters: {'criterion': 'entropy', 'max_depth': 3, 'max_features': 6, 'min_samples_leaf': 5}
Best score is 0.7513020833333334


RandomizedSearchCV do not outperform GridSearchCV. Random Search is important because it saves on computation time.